# Streaming: Processing Unlimited Frames On-Disk

A key feature of trackpy is the ability to process an unlimited number of frames.

For feature-finding, this is straightforward: a frame is loaded, features are located, the locations are saved the disk, and the memory is cleared for the next frame. For linking, the problem is more challenging, but trackpy handles all this complexity for you, using as little memory as possible throughout.

When data sets become large, beginning-friendly file formats like CSV or Excel become impractical. We recommend using the [HDF5 file format](https://support.hdfgroup.org/HDF5/), which is trackpy can read and write out of the box. (HDF5 is [widely used](http://en.wikipedia.org/wiki/Hierarchical_Data_Format); you can be sure it will be around for many, many years to come.)

If you have some other format in mind, see the end of this tutorial, where we explain how to extend trackpy's interface to support other formats.

## Install PyTables

You need pytables, which you can easily install using conda. (Type this command into a Terminal or Command Prompt.)

    conda install pytables
    
## Locate Features, Streaming Results into an HDF5 File

In [1]:
import trackpy as tp
import pims

In [2]:
def gray(image):
    return image[:, :, 0]

images = pims.ImageSequence('../sample_data/bulk_water/*.png', process_func=gray)
images = images[:10]  # We'll take just the first 10 frames for demo purposes.

In [3]:
# For this demo, we'll first remove the file if it already exists.
!rm -f data.h5

We can use `locate` inside a loop:

In [4]:
with tp.PandasHDFStore('data.h5') as s:  # This opens an HDF5 file. Data will be stored and retrieved by frame number.
    for image in images:
        features = tp.locate(image, 11, invert=True)  # Find the features in a given frame.
        s.put(features)  # Save the features to the file before continuing to the next frame.

or, equivalently, we can use `batch`, which accepts the storage file as `output`.

In [5]:
with tp.PandasHDFStore('data.h5') as s:
    tp.batch(images, 11, invert=True, output=s)

Frame 9: 0 features


We can get the data for a given frame:

In [6]:
with tp.PandasHDFStore('data.h5') as s:
    frame_2_results = s.get(2)
    
frame_2_results.head()  # Display the first few rows.

,x,y,mass,size,ecc,signal,raw_mass,ep,frame
0,296.181623,5.629994,1.317008,2.502795,0.232035,0.065567,35.125488,0.104669,2
1,68.334140,6.232143,1.337245,2.706093,0.089628,0.048568,36.349017,0.069776,2
2,336.808777,6.440439,1.032884,2.256825,0.052063,0.061520,34.568625,0.135511,2
3,431.767517,6.907273,2.125669,2.995045,0.412907,0.064353,36.043135,0.076120,2
4,460.814096,7.609493,1.125569,2.713518,0.097293,0.043307,34.960782,0.112224,2


Or dump all the data, if your machine has enough memory to hold it:

In [7]:
with tp.PandasHDFStore('data.h5') as s:
    all_results = s.dump()
    
all_results.head()  # Display the first few rows.

,x,y,mass,size,ecc,signal,raw_mass,ep,frame
0,295.084431,5.614319,1.198659,2.443001,0.194062,0.065175,35.070586,0.109599,0
1,431.473526,7.265490,2.156048,2.781234,0.290554,0.071248,36.074508,0.076321,0
2,67.704366,7.671550,1.381636,2.722437,0.110971,0.050197,36.549017,0.066742,0
3,629.144305,8.152268,1.677556,3.269174,0.204030,0.032385,36.827449,0.062164,0
4,36.059651,8.183902,1.554087,2.833534,0.128563,0.048173,37.376468,0.054758,0


You can dump the first N frames using `s.dump(N)`.

## Link Trajectories, Streaming From and Updating the HDF5 File

In [8]:
with tp.PandasHDFStore('data.h5') as s:
    for linked in tp.link_df_iter(s, 3, neighbor_strategy='KDTree'):
        s.put(linked)

Frame 9: 318 trajectories present


The original data is overwritten.

In [9]:
with tp.PandasHDFStore('data.h5') as s:
    frame_2_results = s.get(2)
    
frame_2_results.head()  # Display the first few rows.

,x,y,mass,size,ecc,signal,raw_mass,ep,frame,particle
0,296.181623,5.629994,1.317008,2.502795,0.232035,0.065567,35.125488,0.104669,2,0.0
1,431.767517,6.907273,2.125669,2.995045,0.412907,0.064353,36.043135,0.076120,2,1.0
2,68.334140,6.232143,1.337245,2.706093,0.089628,0.048568,36.349017,0.069776,2,2.0
3,629.032908,8.015472,1.648082,3.260204,0.210865,0.032379,36.760782,0.062738,2,3.0
4,36.097756,8.154647,1.515328,2.819110,0.136879,0.047354,37.372547,0.054561,2,4.0


## Framewise Data Interfaces

### Built-in interfaces

There are three different interfaces. You can use them interchangeably. They offer different performance advantages.

* [`PandasHDFStore`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L103-L161) -- fastest for a small (~100) number of frames
* [`PandasHDFStoreBig`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L164-L227) -- fastest for a medium or large number of frames
* [`PandasHDFStoreSingleNode`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L230-L320) -- optimizes HDF queries that access multiple frames (advanced)

### Writing your own interface

Trackpy implements a generic interface that could be used to store and retrieve particle tracking data in any file format. We hope that it can make it easier for researchers who use different file formats to exchange data. Any in-house format could be accessed using the same simple interface demonstrated above.

At present, the interface is implemented only for HDF5 files. To extend it to any format, write a class subclassing [`trackpy.FramewiseData`](https://github.com/soft-matter/trackpy/blob/c309b83296c720b8b9fd9633259c2b9ea6740eeb/trackpy/framewise_data.py#L14-L86). This custom class must implement the methods `put`, `get`, `close`, and `__iter__` and the properties `max_frame` and `t_column`. Refer to the built-in classes in [framewise_data.py](https://github.com/soft-matter/trackpy/blob/master/trackpy/framewise_data.py) for examples to work from.